In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import sys
from datetime import datetime as dt

In [2]:
# read in upd_gen csv
upd_gen_df = pd.read_csv('a_upd_generator_df.csv')
bess_upd_gen_df = upd_gen_df.copy()

In [3]:
# load utility-scale battery storage data
utility_scale_battery_df = bess_upd_gen_df[bess_upd_gen_df['Resource'].str.contains('Utility-Scale Battery Storage', na=False)]
print(utility_scale_battery_df)

    Unnamed: 0                              Resource  Model  New_Build  \
10          11  Utility-Scale Battery Storage - 10Hr    NaN        NaN   
11          12   Utility-Scale Battery Storage - 2Hr    NaN        NaN   
12          13   Utility-Scale Battery Storage - 4Hr    NaN        NaN   
13          14   Utility-Scale Battery Storage - 6Hr    NaN        NaN   
14          15   Utility-Scale Battery Storage - 8Hr    NaN        NaN   

    Can_Retire  Zone  THERM  MUST_RUN  STOR  FLEX  ...  MinCapTag_2  \
10         NaN   NaN    NaN       NaN   NaN   NaN  ...          NaN   
11         NaN   NaN    NaN       NaN   NaN   NaN  ...          NaN   
12         NaN   NaN    NaN       NaN   NaN   NaN  ...          NaN   
13         NaN   NaN    NaN       NaN   NaN   NaN  ...          NaN   
14         NaN   NaN    NaN       NaN   NaN   NaN  ...          NaN   

    MinCapTag_3  MGA  Resource_Type  CapRes_1  ESR_1  ESR_2  region  cluster  \
10          NaN  NaN            NaN       NaN   

In [4]:
reqd_bess_data = [
                #   'Zone',
                #   'Model',
                #   'New_Build',
                #   'Can_Retire',
                #   'Existing_Cap_MW',
                #   'Max_Cap_MW',
                #   'Max_Cap_MWh',
                #   'Min_Cap_MW',
                  # 'Min_Cap_MWh',
                  'Inv_Cost_per_MWyr',
                  'Inv_Cost_per_MWhyr',
                  'Fixed_OM_Cost_per_MWyr',
                  'Fixed_OM_Cost_per_MWhyr',
                  'Var_OM_Cost_per_MWh',
                  'Var_OM_Cost_per_MWh_In',
                  'Self_Disch',
                  'Eff_Up',
                  'Eff_Down',
                  'Min_Duration',
                  'Max_Duration',
                #   'Reg_Max',
                #   'Rsv_Max',
                #   'Reg_Cost',
                #   'Rsv_Cost',
                #   'region',
                #   'cluster',
]

In [5]:
import textwrap

# for each battery, check that all required data is present
missing_data = {}

for index, row in utility_scale_battery_df.iterrows():
    missing_columns = [col for col in reqd_bess_data if pd.isna(row[col])]
    if missing_columns:
        missing_data[row['Resource']] = missing_columns

for resource, columns in missing_data.items():
    print(resource)
    print(textwrap.fill(", ".join(columns), width=70))
    print()

Utility-Scale Battery Storage - 10Hr
Inv_Cost_per_MWhyr, Fixed_OM_Cost_per_MWhyr, Var_OM_Cost_per_MWh_In,
Self_Disch, Eff_Up, Eff_Down, Min_Duration, Max_Duration

Utility-Scale Battery Storage - 2Hr
Inv_Cost_per_MWhyr, Fixed_OM_Cost_per_MWhyr, Var_OM_Cost_per_MWh_In,
Self_Disch, Eff_Up, Eff_Down, Min_Duration, Max_Duration

Utility-Scale Battery Storage - 4Hr
Inv_Cost_per_MWhyr, Fixed_OM_Cost_per_MWhyr, Var_OM_Cost_per_MWh_In,
Self_Disch, Eff_Up, Eff_Down, Min_Duration, Max_Duration

Utility-Scale Battery Storage - 6Hr
Inv_Cost_per_MWhyr, Fixed_OM_Cost_per_MWhyr, Var_OM_Cost_per_MWh_In,
Self_Disch, Eff_Up, Eff_Down, Min_Duration, Max_Duration

Utility-Scale Battery Storage - 8Hr
Inv_Cost_per_MWhyr, Fixed_OM_Cost_per_MWhyr, Var_OM_Cost_per_MWh_In,
Self_Disch, Eff_Up, Eff_Down, Min_Duration, Max_Duration



In [6]:
# assume costs are accounted for in MWyr instead of MWhyr
bess_upd_gen_df['Inv_Cost_per_MWhyr'] = 0
bess_upd_gen_df['Fixed_OM_Cost_per_MWhyr'] = 0
bess_upd_gen_df['Var_OM_Cost_per_MWh_In'] = 0
bess_upd_gen_df['Self_Disch'] = 0


In [7]:
# assume efficiency is 85%
bess_upd_gen_df['Eff_Up'] = 0.85
bess_upd_gen_df['Eff_Down'] = 0.85

In [8]:
# loop through each battery and set min duration and max duration equal to the number in their name
for index, row in bess_upd_gen_df.iterrows():
    if 'Battery Storage' in row['Resource']:
        duration = int(row['Resource'].split('-')[-1].replace('Hr', ''))
        bess_upd_gen_df.at[index, 'Min_Duration'] = duration
        bess_upd_gen_df.at[index, 'Max_Duration'] = duration

In [9]:
# check that no batteries are missing data
missing_data = {}
# select only battery storage resources
second_check_bess_df = bess_upd_gen_df[bess_upd_gen_df['Resource'].str.contains('Battery Storage', na=False)]

for index, row in second_check_bess_df.iterrows():
    missing_columns = [col for col in reqd_bess_data if pd.isna(row[col])]
    if missing_columns:
        missing_data[row['Resource']] = missing_columns

if not missing_data:
    print("No batteries are missing data.")
else:
    for resource, columns in missing_data.items():
        print(resource)
        print(textwrap.fill(", ".join(columns), width=70))
        print()

No batteries are missing data.


In [10]:
bess_upd_gen_df.to_csv('a_upd_generator_df.csv', index=False)